# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class': 'SimpleStrategy', 'replication_factor' : 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# PART II. Model and insert data into Apache Cassandra tables and run your queries

## Question: 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

### Answer: 1. For this I will be creating table 'music_app_session_history'.

For table music_app_session_history, I have created a composite primary key using the session_id and the item_in_session. The session_id was used as a partition key because the queries will filter by this column. item_in_session was used as clustering column to help make up a unique key.

In [8]:
# Read the csv file to a pandas dataframe and see what the resulting dataframe should look like
df0 =  pd.read_csv("event_datafile_new.csv")
df0.loc[(df0['sessionId'] == 338) & (df0['itemInSession'] == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2949,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [9]:
# Check the data types of each column
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_app_session_history"
query = query + "(session_id int, item_in_session int, artist text, song text, song_length float, \
                    PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_session_history (session_id, item_in_session, artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (
           int(line[8]),int(line[3]) , line[0], line[9], float(line[5])
        ))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist, song, song_length FROM music_app_session_history WHERE session_id = 338 AND item_in_session = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Question 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Answer: 2. For this I will be creating table 'music_app_artist_history'.

For table music_app_artist_history, I need to filter by two columns user_id and session_id.
I also need to sort the results by item_in_session. 
Therefore I have created a composite primary key using the user_id, session_id and item_in_session.
The PRIMARY KEY is like this ( (user_id, session_id), item_in_session)) because it improves overall performance 
The userId data will be spread among more than just one node and it'll be much faster to look for a specific session_id.

In [13]:
# Read the csv file to a pandas dataframe and see what the resulting dataframe should look like
df =  pd.read_csv("event_datafile_new.csv")
df.loc[(df['userId'] == 10) & (df['sessionId'] == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5000,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
5001,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
5002,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
5003,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [14]:
# Check the data types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query = "CREATE TABLE IF NOT EXISTS music_app_artist_history"
query = query + "(user_id int, session_id int, item_in_session int, \
                    artist text, first_name text,  last_name text, song text, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_artist_history (user_id, session_id, item_in_session, \
                    artist, first_name, last_name, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (
           int(line[10]) , int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]
        ))

        
# Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist, song, first_name, last_name FROM music_app_artist_history \
            WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)                   

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Question: 3 Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Answer: 3. For this I will be creating table 'music_app_song_history'.
For table music_app_song_history, I have created a composite primary key using the song and the user_id. The song was used as a partition key because the queries will filter by this column. 
user_id was used as clustering column to help make up a unique key.

In [16]:
# Read the csv file to a pandas dataframe and see what the resulting dataframe should look like
df1 =  pd.read_csv("event_datafile_new.csv")
df1.loc[(df1['song'] == 'All Hands Against His Own')]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
3503,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
5099,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
6773,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


In [17]:
query = "CREATE TABLE IF NOT EXISTS music_app_song_history"
query = query + "(song text, user_id int, first_name text, last_name text, session_id int, \
                    PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_song_history (song, user_id, first_name, last_name, session_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4], int(line[8])))
        
# Add in the SELECT statement to verify the data was entered into the table

query = "SELECT first_name, last_name FROM music_app_song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)                   
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
query = "DROP TABLE IF EXISTS music_app_session_history"

try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS music_app_artist_history"

try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS music_app_song_history"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()